# Efficient walk circuit for Metropolis-Hastings

This one is based on [https://arxiv.org/pdf/1910.01659](https://arxiv.org/pdf/1910.01659) Section 2 and Appendix A. 

## Generating Boltzmann distributions

Given a discrete configuration space $X=\{+1,-1\}^n$ and an energy function $E(x)$, the Boltzmann (Gibbs) distribution at inverse temperature $\beta$ is the probability mass function $\pi_\beta(x)=Z(\beta)^{-1}e^{-\beta E(x)}$, where $Z(\beta)=\sum_{x\in X}e^{-\beta E(x)}$ is the partition function. 

### The Ising model

In the paper’s setting, $E$ is a $(k,d)$-local Ising model (Eq. 22),
$$
E(x)=\sum_{\ell = 1}^m J_{\ell}\prod_{s\in\Omega_{\ell},} x_s, \qquad |\Omega_{\ell}| \le k.
$$
where each interaction set $\Omega_{\ell}$ has size at most $k$, and each spin participates in interactions involving at most $d$ other spins. 

**Condition on $d$**: the paper states “and each spin interacts with at most $d$ other spins”. I'm not really sure which of these two conditions they refer to:

* Constraint on the term-incidence bound (hypergraph degree), $\max_{s \in [n]} \lvert\{ \ell \mid s \in \Omega_\ell \}\rvert \le d$, i.e. every spin $s$ is included in at most $d$ terms.

    I am inclined to believe this one, as they later mention the number of terms ranges in $m = 1, ..., \frac{nd}{k}$ as this bound follows by a standard double-counting argument: the total number of spin–term incidences is $\sum_{\ell=1}^m |\Omega_\ell|$, which is at most $\sum_{s=1}^n d = nd$ under the incidence constraint, while also $\sum_{\ell=1}^m |\Omega_\ell| = mk$ when each term has size $k$, yielding $mk \le nd$ and hence $m \le \frac{nd}{k}$.

* Constraint on the neighbor-degree bound (degree in the induced interaction graph), $\max_{s \in [n]} \bigl|\{ t \ne s \mid \exists \ell \text{ such that } {s,t} \subseteq \Omega_\ell\}\bigr| \le d$, i.e. every spin has at most $d$ distinct interaction partners across all terms.

    This condition by itself does not imply $m \le nd/k$ for general $k>2$; it only bounds how many different spins a given spin can couple to, not how many distinct $k$-body terms may be formed among those neighbors.

**Example**: A special case is the pairwise model ($k=2$):
$$
E(x)=\sum_{(i,j)\in \mathcal{G}}J_{ij}x_ix_j+\sum_i h_i x_i,
$$
with $\mathcal{G}$ a graph of maximum degree $d$. 

**Description size**: The description size of a $(k,d)$-local Ising model is polynomial in $n, k, d$: you specify the coefficients $J_\ell$ and the supports $\Omega_\ell$ (note $m \le nd/k$). 

### The use of Metropolis–Hastings

The transition matrix $P$ can be huge ($2^n \times 2^n$ for $n$ spins). We would like:
1. to efficiently compute (nonzero) entries of $P$ without materializing the full matrix;
2. to satisfy detailed balance with respect to the Boltzmann distribution $\pi_\beta(x) \propto e^{-\beta E(x)}$.

A standard way to achieve both is to define $P$ via a Metropolis–Hastings proposal-and-accept mechanism: first propose a candidate move using a simple sparse kernel $T$, then accept/reject it using an acceptance rule $A$ chosen to enforce detailed balance.

#### Matrix $T$ (proposal kernel)

The matrix $T$ is chosen to make proposals easy to sample and sparse. Instead of allowing all possible moves from one state $x$ to another state $y$, we allow only a small number $N$ of moves. In the $\{\pm 1\}^n$ encoding, a move is a string $z\in\{\pm 1\}^n$ and applying the move means flipping the spins where $z_s=-1$, i.e. $y = x \cdot z$ (componentwise product).

For example, “flip a single spin” corresponds to $N=n$ moves: $z^{(i)}\in\{\pm 1\}^n$ with $z^{(i)}_i=-1$ and $z^{(i)}_s=+1$ for $s\neq i$. For $n=4$ these are
$$
(-,+,+,+),\ (+,-,+,+),\ (+,+,-,+),\ (+,+,+,-).
$$
If we choose these moves uniformly, then
$$
[T]_{y x} = [T]_{x y} =
\begin{cases}
\frac{1}{n}, & x \cdot y = z \text{ with exactly one }(-1)\text{ entry},\\
0, & \text{otherwise.}
\end{cases}
$$

More generally, if we allow a set of moves $\mathcal{M}=\{z_j\}_{j=1}^N$ and choose them uniformly at random, we have
$$
[T]_{y x} = [T]_{x y} =
\begin{cases}
\frac{1}{N}, & x \cdot y \in \mathcal{M},\\
0, & \text{otherwise.}
\end{cases}
$$

Thus $T$ is row-sparse: for each fixed $x$, there are at most $N$ states $y$ with $[T]_{y x}\neq 0$. We do **not** enumerate the $2^n\times N$ matrix, as this is still unfeasible. The entries can be calculated on the spot when needed: we only generate $y$ from $x$ by sampling a move $z\in\mathcal{M}$ and setting $y=x\cdot z$.

#### Matrix $A$ (acceptance rule)

To satisfy detailed balance with respect to $\pi_\beta(x)\propto e^{-\beta E(x)}$, Metropolis–Hastings accepts a proposed move $x\to y$ with probability
$$
A_{y x} = \min\!\left(1,\ \frac{\pi_\beta(y)\,[T]_{x y}}{\pi_\beta(x)\,[T]_{y x}}\right).
$$
Since we have conveniently chosen $T$ to be symmetric, $[T]_{x y}=[T]_{y x}$, this simplifies to
$$
A_{y x} = \min\!\left(1,\ \frac{\pi_\beta(y)}{\pi_\beta(x)}\right)
= \min\!\left(1,\ e^{-\beta(E(y)-E(x))}\right)
= \min\!\left(1,\ e^{\beta(E(x)-E(y))}\right).
$$

In practice one computes $\Delta = E(y)-E(x)$; if $\Delta\le 0$ the move is always accepted, and if $\Delta>0$ it is accepted with probability $e^{-\beta\Delta}$. 

**Is this true?:** For $(k,d)$-local Ising models and local moves (e.g. single-spin flips), $\Delta$ depends only on the terms that touch the flipped spins, so it can be computed without evaluating $E(\cdot)$ globally.

#### The resulting transition matrix $P$

The Metropolis–Hastings transition probabilities are then
$$
P_{y x} =
\begin{cases}
[T]_{y x}\,A_{y x}, & y\neq x,\\[4pt]
1-\sum_{y\neq x}[T]_{y x}\,A_{y x}, & y=x,
\end{cases}
$$
i.e. you propose $y$ using $T$, accept with probability $A_{y x}$, and otherwise remain at $x$. With the acceptance rule above, $P$ satisfies detailed balance $\pi_\beta(x)P_{y x}=\pi_\beta(y)P_{x y}$, hence $\pi_\beta$ is a stationary distribution of the chain.



## Costly, arithmetic-based implementation of $W$

Let $P$ be the classical Metropolis–Hastings transition matrix on a state space of size $d$, with entries $[P]_{y x}=P_{y x}$. We seek to implement the Szegedy walk oracle $W$, i.e. a unitary on $\mathcal{H}=\mathbb{C}^d\otimes\mathbb{C}^d$ such that, for every basis state $|x\rangle$,
$$
W\big(|x\rangle_L|0\rangle_R\big)=|x\rangle_L\Big(\sum_{y=0}^{d-1}\sqrt{P_{y x}}\,|y\rangle_R\Big),
$$
up to an arbitrary completion on the orthogonal subspace. 

In the Ising-model setting one typically has $d=|X|=2^n$, where $n$ is the number of spins (hence $n$ qubits are used to represent a configuration $x\in\{\pm1\}^n$). Appendix A explains why implementing this oracle directly is costly: while the off-diagonal amplitudes can be generated from local proposals and acceptance probabilities, completing the diagonal amplitude requires a global, arithmetic-heavy coherent computation.

### Logical and auxiliary registers

A direct construction uses both **logical registers** (which represent the input/output of the oracle) and **auxiliary workspace registers** (which must be returned to $|0\rangle$ at the end to obtain a clean unitary implementation of the oracle):

* **Left system register $L$** ($n$ qubits): holds the current configuration $x$ as $|x\rangle_L$. This register is preserved by the oracle (it labels the column of $P$ being loaded into amplitudes).

* **Right system register $R$** ($n$ qubits): holds the candidate next configuration. It starts in $|0\rangle_R$ and is the register on which the superposition $\sum_y \sqrt{P_{y x}}|y\rangle_R$ is prepared.

* **Move register $M$** ($\lceil\log_2 N\rceil$ qubits for binary encoding, or $N$ qubits for unary encoding): stores which proposal move was selected (e.g., a single-spin flip index). It is a workspace register used to construct the proposal distribution and must be uncomputed back to $|0\rangle_M$. The papr uses the unary encoding as it results in shallower circuits. 

* **Coin / acceptance register $C$** (1 qubit, often plus auxiliary space): stores the accept/reject amplitude via a controlled rotation
  $$
  |0\rangle_C \mapsto \sqrt{1-A_{y x}}\,|0\rangle_C+\sqrt{A_{y x}}\,|1\rangle_C,
  $$
  where $A_{y x}$ is the Metropolis–Hastings acceptance probability for the proposed move. This register (and any scratch used to compute $A_{y x}$) must be uncomputed to return to $|0\rangle$.

Optionally (and this is where the cost concentrates), one introduces additional **arithmetic ancillas** to compute energy differences $\Delta=E(y)-E(x)$, exponentials such as $e^{-\beta\Delta}$, comparisons/min operations, and controlled rotation angles; all such ancillas must also be uncomputed for the overall mapping to be a valid oracle $W$.


### Algorithm

#### The move preparation $V$ 

You first fix a finite set of allowed moves $\mathcal M=\{z_1,\dots,z_N\}\subset\{\pm1\}^n$. Applying a move $z_j$ to a configuration is componentwise multiplication $y = x\cdot z_j$. The moves can be chosen with uniform probability or in general with a probability distribution $f$ supported on $\mathcal M$. The unitary $V$ is a state-preparation unitary for the Move register $M$ that encodes $f$ in amplitudes:
$$
V:\ |0\rangle_M \longmapsto |f\rangle_M := \sum_{j=1}^N \sqrt{f(z_j)}\,|z_j\rangle_M.
$$

**Simple "single spin-flip moves" example**: we have $N = n$ moves $z_j$ flipping the spin $j$, with $f(z_j) = 1/n$. Here $\ket{f}_M = \frac{1}{\sqrt n} \sum_{0}^{n-1} \ket{2^i}_m$. 

* In case $n = 2^m$ this can be prepared by:
    1) start with $100\cdots 0$
    2) apply a "binary tree "of $\sqrt{\text{SWAP}}$ gates: these split $\ket{10}$ into a uniform superposition of $\ket{10}$ and $\ket{01}$. 

* In case $n \neq 2^m$ pad to the next power of two

#### The proposal update $F$ (spin flip)

Given the Move register $M$ prepared in $|f\rangle_M$, we need to coherently map the current configuration $x$ to the proposed configuration $y=x\cdot z_j$ conditioned on the selected move. Introduce a unitary $F$ acting on $(R,M)$ such that
$$
F:\ |x\rangle_R|z_j\rangle_M \longmapsto |x\cdot z_j\rangle_R|z_j\rangle_M.
$$
Operationally, $F$ flips exactly those spins in $R$ for which $z_{j,s}=-1$, controlled on the move label being $z_j$.

**Simple "single spin-flip moves" example**: $M$ is unary with basis states $|j\rangle_M$ and $z_j$ flips spin $j$. Then $F$ applies a controlled-$X$ on $R_j$ controlled by $M_j$ for each $j\in\{1,\dots,n\}$. This is particularly simple in unary because the control for move $j$ is just the single qubit $M_j$, and is the reason why the $M$ register is not chosen in binary. 

#### The Boltzmann coin $B$

After proposing $y$ in the Right register, we must coherently encode the Metropolis–Hastings acceptance probability into a single “coin” qubit. Introduce a unitary $B$ acting on $(L,R,C)$ such that
$$
B:\ |x\rangle_L|y\rangle_R|0\rangle_C \longmapsto
|x\rangle_L|y\rangle_R\Big(\sqrt{1-A_{yx}}\,|0\rangle+\sqrt{A_{yx}}\,|1\rangle\Big)_C,
$$
where for symmetric proposals $A_{yx}$ is
$$
A_{yx}=\min\!\big(1,e^{\beta(E(x)-E(y))}\big).
$$

This step require arithmetics though so that's the most complicated one (so far).

#### Procedure

We consider the simplest case of single spin-flip moves with unary encoding of $M$.

1. Start in the state
$$
|x\rangle_L\,|0\rangle_R\,|0\rangle_M\,|0\rangle_C.
$$

2. Copy $x$ into the right register $R$ using $n$ CNOTs:
$$
|x\rangle_L\,|x\rangle_R\,|0\rangle_M\,|0\rangle_C.
$$

3. Apply $V$ (uniform superposition over moves):
$$
|x\rangle_L\,|x\rangle_R\left(\frac{1}{\sqrt{n}}\sum_{i=1}^n |i\rangle_M\right)|0\rangle_C.
$$

4. Apply the proposal update $F$ (controlled on $M$ apply the selected move to $R$):
$$
\frac{1}{\sqrt{n}}\sum_{i=1}^n |x\rangle_L\,|x\cdot i\rangle_R\,|i\rangle_M\,|0\rangle_C.
$$

5. Apply the Boltzmann coin $B$ (controlled rotation on the coin so that the $\ket{1}$ probability equals the MH acceptance):
$$
\frac{1}{\sqrt{n}}\sum_{i=1}^n |x\rangle_L\,|x\cdot i\rangle_R\,|i\rangle_M
\left(\sqrt{1-A_{(x\cdot i)x}}\,|0\rangle_C+\sqrt{A_{(x\cdot i)x}}\,|1\rangle_C\right).
$$

6. Apply a controlled-SWAP controlled by $C$ that swaps $L$ and $R$ when $C=1$. This yields
$$
\begin{align*}
& \frac{1}{\sqrt{n}}\sum_{i=1}^n |x\rangle_L\,|x\cdot i\rangle_R\,|i\rangle_M\,\sqrt{1-A_{(x\cdot i)x}}\,|0\rangle_C
\;+\;
\frac{1}{\sqrt{n}}\sum_{i=1}^n |x\cdot i\rangle_L\,|x\rangle_R\,|i\rangle_M\,\sqrt{A_{(x\cdot i)x}}\,|1\rangle_C \\
=\,\,& \sum_{i=1}^n |x\rangle_L\,|x\cdot i\rangle_R\,|i\rangle_M\,\sqrt{T_{(x\cdot i)x}(1-A_{(x\cdot i)x})}\,|0\rangle_C
\;+\;
\sum_{i=1}^n |x\cdot i\rangle_L\,|x\rangle_R\,|i\rangle_M\,\sqrt{T_{(x\cdot i)x}A_{(x\cdot i)x}}\,|1\rangle_C \\
=\,\,& \sum_{i=1}^n |x\rangle_L\,|x\cdot i\rangle_R\,|i\rangle_M\,\sqrt{T_{(x\cdot i)x}(1-A_{(x\cdot i)x})}\,|0\rangle_C
\;+\;
\sum_{i=1}^n |x\cdot i\rangle_L\,|x\rangle_R\,|i\rangle_M\,\sqrt{P_{(x\cdot i)x}}\,|1\rangle_C,
\end{align*}
$$
where in this uniform single-flip case $T_{(x\cdot i)x}=1/n$ and $P_{(x\cdot i)x}=T_{(x\cdot i)x}A_{(x\cdot i)x}$. The accept branch already has the desired ordering $|y\rangle_L|x\rangle_R$ while the reject branch keeps $|x\rangle_L$ and leaves $|y\rangle_R$ in superposition.

7. Uncompute the Move register and discard it. In this unary single-flip setting, the move index $i$ is determined by comparing the two system registers; however the ordering differs between branches, so the uncomputation must be controlled on $C$:
    - if $C=0$ (reject) recover $i$ from $(L,R)=(x,y)$,
    - if $C=1$ (accept) recover $i$ from $(L,R)=(y,x)$.
   
    Appendix A counts this as $O(2N)$ CNOT-type operations in unary. After resetting $M\to|0\rangle_M$ we obtain
    $$
    \sum_{i=1}^n |x\rangle_L\,|x\cdot i\rangle_R\,\sqrt{T_{(x\cdot i)x}(1-A_{(x\cdot i)x})}\,|0\rangle_C
    \;+\;
    \sum_{i=1}^n |x\cdot i\rangle_L\,|x\rangle_R\,\sqrt{P_{(x\cdot i)x}}\,|1\rangle_C.
    $$

8. **(Important here)** Complete the diagonal amplitude by a unitary that depends on $x$. Define a family of unitaries $\{G_x\}$ on $R$ such that
$$
G_x:\ \sum_{y\neq x}\sqrt{T_{yx}(1-A_{yx})}\,|y\rangle_R \longmapsto \sqrt{P_{xx}}\,|x\rangle_R,
\qquad
P_{xx}=1-\sum_{y\neq x}P_{yx}.
$$
In the circuit this appears as a controlled operation that applies $G_x$ on $R$ conditioned on $C=0$ and on the value stored in $L$:
$$
G := \sum_{x}|x\rangle\langle x|_L\otimes\big(|0\rangle\langle 0|_C\otimes G_x + |1\rangle\langle 1|_C\otimes I_R\big).
$$
This is a costly arithmetic-based construction, because $G_x$ depends on all acceptance coefficients $\{A_{yx}\}_{y\neq x}$. You also need a clever way to syntetize $G$ directly and avoid composing it from the exponentially many $G_x$. 

9. Applying $G$ maps the reject superposition on $R$ to the diagonal amplitude, giving
$$
\sqrt{P_{xx}}\,|x\rangle_L\,|x\rangle_R\,|0\rangle_C
\;+\;
\sum_{i=1}^n \sqrt{P_{(x\cdot i)x}}\,|x\cdot i\rangle_L\,|x\rangle_R\,|1\rangle_C.
$$
(Here we used $\sum_{y\neq x}T_{yx}(1-A_{yx})=1-\sum_{y\neq x}T_{yx}A_{yx}=P_{xx}$.)

10. Uncompute the coin register $C\to|0\rangle_C$. Note $C$ equals $1$ exactly on the accept branch, which is exactly the branch where $L\neq R$ (since accept implies $L=y\neq x=R$), while in the reject branch $L=R=x$ and $C=0$. Therefore we can reset $C$ by computing a single ancilla flagging $x \neq y$ and applying a CNOT controlled by such ancilla to flip $C$ then uncomputing. This leads to the oracle in the form
$$
\left(\sqrt{P_{xx}}\,|x\rangle_L + \sum_{i=1}^n \sqrt{P_{(x\cdot i)x}}\,|x\cdot i\rangle_L\right)\,|x\rangle_R\,|0\rangle_C
$$

## Cheap, arithmetic-free implementation of $\tilde U_W$

### From the oracle walk $U_W$ to the isometric walk $\tilde U_W$

Here we focus on implementing $U_W$ directly instead of $W$. This is ok, we can equivalently use the half-step $U_W$ without major issues. We recall $U_W$ acts on the Hilbert space $\mathcal{H} = \mathcal{C}^d \otimes \mathcal{C}^d$ which in our case, with $d = 2^n$, we have $\mathcal{H} \cong \mathcal{C}^{2n}$. 

The authors find more convenient to change the encoding of the transformation. Instead of working with pair $\ket{x}_L\ket{y}_R \in \mathcal{H}$ of current and next states, we can focus on pairs $\ket{x}\ket{z} \in \mathcal{C}^d \otimes \mathrm{span}(\mathcal{M})$ of current state and move, with the move $z = x \cdot y$. 
This change of representation is formalized by a partial isometry $Y$, and the implementable walk is defined by
$$\tilde U_W := Y^\dagger U_W Y.$$
Concretely, $Y$ is specified on computational basis states by
$$
Y:\ |x\rangle_L\otimes|y\rangle_R \longmapsto
\begin{cases}
|x\rangle_S\otimes|x\cdot y\rangle_M, & \text{if } x\cdot y\in\mathcal{M},\\
0, & \text{otherwise,}
\end{cases}
$$

Because $\tilde U_W$ is defined via this isometry, it lives on a different effective space than $U_W$; instead, it reproduces the desired walk dynamics on the subspace corresponding to valid move transitions.

### Registers and encoding (System / Move / Coin)

The circuit implementation of $\tilde U_W$ uses three registers:

- **System register $S$:** stores a configuration $x\in\{\pm1\}^n$ using $n$ qubits, basically it corresponds to the old register $L$;
- **Move register $M$:** stores a move label $j\in\{1,\dots,N\}$. We again encode the move in unary using $N$ qubits, as done previously and for the same reasons.
- **Coin register $C$:** a single qubit $b\in\{0,1\}$.

Thus $\tilde U_W$ is implemented on $n+N$ qubits (the coin register is uncomputed and discarded), while $U_W$ is an oracle-space operator on $2n$ qubits.

### Definition of $\tilde U_W$ (Sec 2 Eq 25–29)

The walk as a product of four unitaries:
$$
\tilde U_W = R\,V^\dagger\,B^\dagger\,F\,B\,V.
$$

The components are:

1. The unitary $V$ (move preparation). $V$ prepares the Move register in a superposition that encodes the proposal distribution $f$ over the allowed moves:
    $$|0\rangle_M \mapsto \sum_{j=1}^N \sqrt{f(j)}\,|j\rangle_M.$$
    In the common uniform case $f(j)=1/N$, this is the uniform one-hot superposition on $M$. We refer to the implementation based on binary tree of $\sqrt{\mathrm{SWAP}}$ gates in the special case $N = n = 2^m$ for some $m$, as in the previous section.

2. Unitary $F$ (conditional move application). $F$ applies the selected move to the system register if and only if the coin indicates acceptance:
$$
|x\rangle_S \ket{z_j}_M \ket{b}_C \mapsto
\begin{cases}
|x\rangle_S \ket{z_j}_M \ket{b}_C, & b=0,\\
|x\cdot z_j\rangle_S \ket{z_j}_M \ket{b}_C, & b=1.
\end{cases}
$$
This is the step that prevents rejected proposals from creating an “unwanted” superposition over neighboring states: in the reject branch the system remains at $x$. You can implement this with $O(N)$ C-CNOT in the single spin-flip model.

3. The unitary $R$ (reflection). $R$ flips the sign of the single state $|0\rangle_M|0\rangle_C$ and acts as identity on all other basis states. Instead of using the usual, linear-depth circuit they propose an alternative scheme achieving logaritmic depth in $n$ at the cost of linearly many ancillas. 

4. The unitary $B$ (Boltzmann coin). $B$ computes the acceptance probability into the amplitudes of the coin qubit, conditioned on the current configuration $x$ (in $S$) and the proposed move $j$ (in $M$):
    $$\ket{x}_S \ket{z_j}_M |0\rangle_C \qquad \mapsto \qquad \ket{x}_S \ket{z_j}_M  \; \Bigg(\sqrt{1-A_{x\cdot z_j,x}}\,|0\rangle_C + \sqrt{A_{x\cdot z_j,x}}\,|1\rangle_C\Bigg).$$

### Details on the Boltzmann coin

#### Example on a $(2,3)$-Ising model

Consider a model allowing interaction between at most $k=2$ elements,
$$
E(x)=\sum_{(u,v)\in\mathcal G} J_{uv}\,x_u x_v \;+\; \sum_{u=1}^n h_u\,x_u,
$$
with $\mathcal G$ being a graph with bounded degree $d=3$. For example $\mathcal G = (\mathcal V = \{1,2,3,4\}, \mathcal E=\{12,13,14,24\})$ and
$$
J_{12} = 0.5, \qquad J_{13} = 0.75, \qquad J_{14} = 0.875, \qquad J_{24} = 0.125, \qquad
h_1 = 0.1, \qquad h_2 = 0.2, \qquad h_3 = 0.3, \qquad h_4 = 0.4.
$$


##### High level structure of $B$

Specifically for the **single spin flip** we can define $B = R_N \cdots R_2 R_1$ as the composition of one rotation per spin flip. Here, $N = |\mathcal V| = 4$ thus $B = R_4 R_3 R_2 R_1$. The unitary $R_j$ is controlled by the move register and it is non-trivial only when it holds value $j$ ($j$-th spin flip in our simplified model, $z_j$ move in the generic one not considered here). 

$$
\begin{align*}
    R_j \; |x\rangle_S |m\rangle_M |0\rangle_C
    & =
    \begin{cases}
    |x\rangle_S |2^j\rangle_M\Big(\sqrt{1-A_{(x\cdot z_j)x}}\,|0\rangle_C+\sqrt{A_{(x\cdot z_j)x}}\,|1\rangle_C\Big), & m=2^j,\\[0.6em]
    |x\rangle_S |m\rangle_M |0\rangle_C, & \text{otherwise,}
    \end{cases}
    \\[1.2em]
    R_1 \; |x\rangle_S |2^0\rangle_M |0\rangle_C
    & =
    |x\rangle_S |2^0\rangle_M\Big(\sqrt{1-A_{(x\cdot z_1)x}}\,|0\rangle_C+\sqrt{A_{(x\cdot z_1)x}}\,|1\rangle_C\Big),\\
    & \text{where }\Delta_1(x)=-2x_1\Big(h_1+J_{12}x_2+J_{13}x_3+J_{14}x_4\Big)
    =-2x_1\Big(0.1+0.5x_2+0.75x_3+0.875x_4\Big),\\
    & \phantom{\text{where }}A_{(x\cdot z_1)x}=\min\!\left(1,e^{-\beta\Delta_1(x)}\right),
    \\[1.2em]
    R_2 \; |x\rangle_S |2^1\rangle_M |0\rangle_C
    & =
    |x\rangle_S |2^1\rangle_M\Big(\sqrt{1-A_{(x\cdot z_2)x}}\,|0\rangle_C+\sqrt{A_{(x\cdot z_2)x}}\,|1\rangle_C\Big),\\
    & \text{where }\Delta_2(x)=-2x_2\Big(h_2+J_{12}x_1+J_{24}x_4\Big)
    =-2x_2\Big(0.2+0.5x_1+0.125x_4\Big),\\
    & \phantom{\text{where }}A_{(x\cdot z_2)x}=\min\!\left(1,e^{-\beta\Delta_2(x)}\right),
    \\[1.2em]
    R_3 \; |x\rangle_S |2^2\rangle_M |0\rangle_C
    & =
    |x\rangle_S |2^2\rangle_M\Big(\sqrt{1-A_{(x\cdot z_3)x}}\,|0\rangle_C+\sqrt{A_{(x\cdot z_3)x}}\,|1\rangle_C\Big),\\
    & \text{where }\Delta_3(x)=-2x_3\Big(h_3+J_{13}x_1\Big)
    =-2x_3\Big(0.3+0.75x_1\Big),\\
    & \phantom{\text{where }}A_{(x\cdot z_3)x}=\min\!\left(1,e^{-\beta\Delta_3(x)}\right),
    \\[1.2em]
    R_4 \; |x\rangle_S |2^3\rangle_M |0\rangle_C
    & =
    |x\rangle_S |2^3\rangle_M\Big(\sqrt{1-A_{(x\cdot z_4)x}}\,|0\rangle_C+\sqrt{A_{(x\cdot z_4)x}}\,|1\rangle_C\Big),\\
    & \text{where }\Delta_4(x)=-2x_4\Big(h_4+J_{14}x_1+J_{24}x_2\Big)
    =-2x_4\Big(0.4+0.875x_1+0.125x_2\Big),\\
    & \phantom{\text{where }}A_{(x\cdot z_4)x}=\min\!\left(1,e^{-\beta\Delta_4(x)}\right).
\end{align*}
$$

##### Local energy difference for a single spin flip ($R_j$)

In this example the dependence of each $R_j$ is local: the angle for $R_j$ depends only on $x_j$ and its graph neighbors. Concretely, $R_1$ depends on $(x_1,x_2,x_3,x_4)$, $R_2$ depends on $(x_1,x_2,x_4)$, $R_3$ depends on $(x_1,x_3)$, and $R_4$ depends on $(x_1,x_2,x_4)$.

Let $z_j\in\{\pm1\}^n$ denote the move that flips only spin $j$. Then only energy terms involving spin $j$ change, and the energy difference is
$$
\Delta_j(x):=E(x\cdot z_j)-E(x)
= -2x_j\left(h_j+\sum_{u\in\mathcal N(j)} J_{ju}x_u\right),
$$
where $\mathcal N(j)=\{u:(j,u)\in\mathcal E\}$ is the neighbor set of $j$ in $\mathcal G$.

More generally, in a $(k,d)$-local model written as $E(x)=\sum_\ell J_\ell\prod_{s\in\Omega_\ell}x_s$ with $|\Omega_\ell|\le k$, flipping spin $j$ only affects the terms with $j\in\Omega_\ell$. The set of spins that $\Delta_j(x)$ can depend on is therefore
$$
N_j=\bigcup_{\ell:\,j\in\Omega_\ell}\Omega_\ell,
$$
so if spin $j$ participates in at most $d$ such terms, then $|N_j|\le 1+d(k-1)\le kd$. In the pairwise case $k=2$ this reduces to $N_j=\{j\}\cup\mathcal N(j)$ and $|N_j|\le d+1$, i.e. the dependence is controlled by the graph degree.

For the concrete graph $\mathcal E=\{12,13,14,24\}$, the neighborhoods are
$$
\mathcal N(1)=\{2,3,4\},\quad \mathcal N(2)=\{1,4\},\quad
\mathcal N(3)=\{1\},\quad \mathcal N(4)=\{1,2\}.
$$
Hence the dependence sets $N_j=\{j\}\cup\mathcal N(j)$ have sizes
$$
|N_1|=4,\quad |N_2|=3,\quad |N_3|=2,\quad |N_4|=3,
$$
which matches the $(2,3)$ locality intuition: the coin rotation for move $j$ depends only on $x_j$ and at most $d=3$ neighbors.


##### Synthesis of $R_j$

Fix a move index $j$ (single-spin flip of spin $j$). The unitary $R_j$ acts non-trivially only when the unary Move register indicates this move, and in that case it applies a coin rotation whose angle depends on the local energy difference:
$$
R_j\,|x\rangle_S|m\rangle_M|0\rangle_C
=
\begin{cases}
|x\rangle_S|2^j\rangle_M\,R_y(2\theta_{x,j})|0\rangle_C, & m=2^j,\\
|x\rangle_S|m\rangle_M|0\rangle_C, & m\neq 2^j,
\end{cases}
$$
where $\sin^2(\theta_{x,j})=A_{(x\cdot z_j)x}$ and (Metropolis–Hastings) $A_{(x\cdot z_j)x}=\min\!\left(1,e^{-\beta\Delta_j(x)}\right)$.

Since $\theta_{x,j}$ depends only on the neighborhood bits $x|_{N_j}$ then $R_j$ can be synthesized as a "*multiplexed rotation*" controlled on those qubits. 

1. Let $|N_j|=t$ (in our example $|N_j| \le 1 + d(k-1) = 1 + 3(2-1) = 4$)
2. Enumerate all $2^t$ assignments $u\in\{0,1\}^t$ of the spins in $N_j$
3. For each $u$, precompute classically the corresponding angle $\theta_{u,j}=\arcsin\!\sqrt{\min\!\left(1,e^{-\beta\Delta_j(u)}\right)}$
4. For each $u$, apply the controlled $R_y$
$$
\text{if }(M=j)\text{ and }(x|_{N_j}=u)\text{ then apply }R_y(2\theta_{u,j})\text{ on }C.
$$


### Table of costs

Let $n$ be the number of spins (system qubits), $N$ the number of allowed moves (size of unary Move register $M$), $t:=\max_j |N_j|$ the maximum neighborhood size needed to compute $\Delta_j(x)$ (for single-spin flips in a $(k,d)$-local model one has $t\le 1+d(k-1)\le kd+1$, and for $k=2$ graphs $t\le d+1$), and let $\epsilon$ be the target synthesis accuracy for each non-Clifford single-qubit rotation.

We count costs in terms of Toffoli-type gates and $T$ gates (Clifford gates are treated as lower order). Depth assumes basic parallel scheduling where gates on disjoint qubits can be executed simultaneously.

| Component | Purpose | Gate count (dominant) | $T$-count (dominant) | Depth (dominant) | Extra ancillas | Notes |
|---|---|---:|---:|---:|---:|---|
| $V$ | Prepare move superposition $|0\rangle_M\mapsto \sum_j \sqrt{f(j)}|j\rangle_M$ | $O(N)$ two-qubit gates | $0$ | $O(\log N)$ | $0$ | For uniform $f$ and $N=2^m$, tree of $\sqrt{\mathrm{SWAP}}$ gates; otherwise pad to next power of 2. |
| $V^\dagger$ | Unprepare move superposition | Same as $V$ | $0$ | Same as $V$ | Same as $V$ | Exact inverse of the above circuit. |
| $F$ | Apply move iff coin accepts: $|x\rangle|j\rangle|b\rangle\mapsto |x\cdot z_j^b\rangle|j\rangle|b\rangle$ | $O(N)$ Toffoli | $O(N)$ (if Toffoli decomposed) | $O(1)$–$O(N)$ | $0$–$O(N)$ | Single-spin flips, unary $M$: one Toffoli per spin $j$ with controls $(M_j,C)$, target $S_j$. Depth can be $O(1)$ with sufficient parallelism/scratch; otherwise $O(N)$. |
| $R$ (standard) | Reflection about $|0\rangle_M|0\rangle_C$ | $O(N)$ Toffoli | $O(N)$ | $O(N)$ | $O(1)$ | Standard multi-controlled phase flip on $N+1$ controls; linear depth without many clean ancillas. |
| $R$ (log-depth variant) | Same reflection as above | $O(N)$ Toffoli | $O(N)$ | $O(\log N)$ | $O(N)$ | Binary-tree compute-and-uncompute of the AND of all controls; trades $O(N)$ ancillas for logarithmic depth. |
| $R_j$ (one move coin rotation) | Conditional coin rotation for move $j$ | $O(2^{|N_j|}\,|N_j|)$ Toffoli | $O(2^{|N_j|}\,\log(1/\epsilon))$ | $O(2^{|N_j|}\,(|N_j|+\log(1/\epsilon)))$ | $O(|N_j|)$ | Implemented as a multiplexor over all $2^{|N_j|}$ patterns of the neighborhood bits, additionally controlled on $(M=j)$. Each branch needs: (i) multi-control plumbing $O(|N_j|)$ Toffoli (with $O(|N_j|)$ clean ancillas), and (ii) one single-qubit $R_y(2\theta)$ synthesized to precision $\epsilon$ costing $O(\log(1/\epsilon))$ $T$ gates. |
| $B$ (sequential) | $B=R_N\cdots R_1$ | $O\!\left(\sum_{j=1}^N 2^{|N_j|}|N_j|\right)$ Toffoli, worst-case $O(N\,2^t\,t)$ | $O\!\left(\sum_{j=1}^N 2^{|N_j|}\log(1/\epsilon)\right)$, worst-case $O(N\,2^t\log(1/\epsilon))$ | Worst-case $O(N\,2^t\,(t+\log(1/\epsilon)))$ | $O(t)$ | Straight-line product: all $R_j$ touch the same coin, so naive scheduling is sequential in $j$. This is typically the dominant cost. |
| $B^\dagger$ | Uncompute coin loading | Same as $B$ | Same as $B$ | Same as $B$ | Same as $B$ | Exact inverse of the above. |
| One step $\tilde U_W$ | $\tilde U_W=R\,V^\dagger B^\dagger F B V$ | $2\cdot\#V + 2\cdot\#B + \#F + \#R$ | $2\cdot T(B)+T(F)+T(R)$ | $\mathrm{depth}(R)+2\,\mathrm{depth}(V)+2\,\mathrm{depth}(B)+\mathrm{depth}(F)$ | Add the maxima used | Use either $R$ implementation (standard or log-depth). In most regimes $B$ dominates unless $t$ is extremely small and $\epsilon$ is very loose. |

**Rotation-synthesis sub-cost (used in $R_j$ and thus $B$):** a single arbitrary $R_y(2\theta)$ to precision $\epsilon$ in Clifford+$T$ typically costs $O(\log(1/\epsilon))$ $T$ gates (constants depend on the synthesis method and allowable ancillas). This factor multiplies the number of multiplexed branches, i.e. $2^{|N_j|}$ per $R_j$.


### Implementation in Qiskit


In [1]:
from dataclasses import dataclass
from itertools import product
import numpy as np
from qiskit.circuit import QuantumCircuit, Gate
from qiskit.circuit.library import RYGate, SwapGate
from qiskit.quantum_info import Statevector, Operator

#### Ising model
Here we allow any $(k,d)$-Ising model but only the single spin-flip move. Gate classes may have different implementation available though they need to be used on the appropriate set of registers (some implementations requires ancillary qubits).

Use a dataclass to make the Ising model immutable.

In [2]:
@dataclass
class IsingModel:
    k: int  # upper bound on the number of spins interacting in a single term
    d: int  # upper bound on the number or terms containing a shared spin
    n: int  # number of spins
    m: int  # number of terms
    J: tuple      # sequence of floats containing the J value for each term
    Omega: tuple  # sequence of tuple[int, ...] containing the spins forming the term

    @staticmethod
    def from_list_clauses(k: int, d: int, n: int, Js: list[float], terms: list[tuple[int,...]]):
        assert 0 < n, "The number of spins n is not positive"
        assert 0 <= d < n, f"Invalid {d=}"
        assert 0 <= k <= n, f"Invalid {k=}"
        assert len(Js) == len(terms) > 0, "Invalid or mismatching number of terms"
        # add terms
        J, Omega = [], []
        for w, om in zip(Js, terms):
            assert len(om) > 0, f"Clause {om} invalid: there must be at least one spin in the interaction term"
            assert len(om) <= k, f"Clause {om} invalid: the number of spin in this term is {len(om)} while the maximum allowed is {k=}"
            assert len(set(om)) == len(om), f"Clause {om} invalid: there are duplicated spins"
            assert all(0 <= i < n for i in om), f"Clause {om} invalid: not all spins are between 0 and {n-1}=n-1"
            J.append(float(w))
            Omega.append(tuple(int(i) for i in sorted(om)))

        # constraint on the number or terms containing a shared spin 
        neigh = [set() for _ in range(n)]
        for om in Omega:
            for spin in om:
                neigh[spin].update(set(om) - {spin})
        if max(len(neigh[spin]) for spin in range(n)) > d:
            raise ValueError("Degree bound d violated.")

        return IsingModel(k=k, d=d, n=n, m=len(Omega), J=tuple(J), Omega=tuple(Omega))

In [3]:
example_model = IsingModel.from_list_clauses(
    k=2, d=3, n=4, 
    Js=[0.1, 0.2, 0.3, 0.4, 0.5, 0.75, 0.875, 0.125],
    terms=[(0,), (1,), (2,), (3,), (0,1), (0,2), (0,3), (1,3)])
example_model

IsingModel(k=2, d=3, n=4, m=8, J=(0.1, 0.2, 0.3, 0.4, 0.5, 0.75, 0.875, 0.125), Omega=((0,), (1,), (2,), (3,), (0, 1), (0, 2), (0, 3), (1, 3)))

#### Unitary $V$

We focus on $V$ preparing the uniform superposition of $N = n$ moves being a single spin flip. Note that here we work in the simplied setting having $N = n = 2^q$ for some $q$. 

We can of course avoid this limitation by padding to the next power of two the moves and consider any move between $N$ and $2^{\lceil \log_2 N\rceil}$ to be a dummy move that stays in the same state, with a slowdown factor of $2^{\lceil \log_2 N\rceil}/N < 2$. This makes the other unitaries a bit more complicated so we skip this for now.

##### A note on the √SWAP implementation

The composition $CX \cdot CSX \cdot CX$ implements a valid $\sqrt{\mathrm{SWAP}}$, but it does not send $|10\rangle$ to $\frac{|10\rangle+|01\rangle}{\sqrt2}$ with equal real amplitudes. Instead (up to a global phase) it maps $\ket{10} \mapsto \frac{\ket{10} + i \ket{01}}{\sqrt 2} $. so the two branches have a relative phase $+\pi/2$.

In the binary-tree construction for $V$, the excitation "moves right" a different number of times depending on which one-hot basis state you end up in. Each "move right" contributes an extra factor of $i$, so without correction you will not get all the phases equal. Probabilities are still uniform, but phases differ, which can break later. The "phase fix" is applying an $S$ gate ($S=\mathrm{diag}(1,i)$) on the right wire after each $\sqrt{\mathrm{SWAP}}$ in the tree. That cancels the $i$ phase picked up when the excitation goes to the right child.

In [4]:
def is_power_of_two(n: int) -> bool:
    return n > 0 and (n & (n - 1)) == 0


class SqrtSwap(Gate):
    def __init__(self, label=None):
        super().__init__("√SWAP", 2, [], label=label)
        self.definition = self._build_definition()

    def _build_definition(self):
        qc = QuantumCircuit(2, name="√SWAP")
        qc.cx(0, 1)
        qc.csx(1, 0)
        qc.cx(0, 1)
        qc.s(1)  # phase-fix so the one-hot amplitudes align
        return qc


class MovePreparationV(Gate):
    def __init__(self, ising_model: IsingModel, label=None):
        self.ising_model = ising_model
        self.n = int(self.ising_model.n)
        if not is_power_of_two(self.n):
            raise ValueError("This simplified implementation works best with n being a power of two")
        super().__init__("V", self.n, [], label=label)
        self.definition = self._build_definition()

    def _build_definition(self):
        qc = QuantumCircuit(self.n, name="V")
        if self.n == 1:
            qc.x(0)
            return qc

        qc.x(0)  # |10..00>

        step = 1
        while step < self.n:
            block = 2 * step
            for start in range(0, self.n, block):
                for j in range(step):
                    a = start + j
                    b = start + j + step
                    qc.append(SqrtSwap(), [a, b])
                    
            step *= 2

        return qc

In [5]:
V = MovePreparationV(example_model)

qc = QuantumCircuit(example_model.n)
qc.append(V, range(example_model.n))

sv = Statevector.from_label("0" * example_model.n).evolve(qc)
probs = sv.probabilities_dict()
probs

{np.str_('0001'): np.float64(0.25),
 np.str_('0010'): np.float64(0.25),
 np.str_('0100'): np.float64(0.25),
 np.str_('1000'): np.float64(0.25)}

### Unitary $F$

TODO need to implement log depth variant.

#### Linear depth implementation

For the **linear** implementation of $F$, we exploit unary encoding of the move register $M$: exactly one qubit $M_j$ is $1$ (meaning “propose flipping spin $j$”). With a single coin qubit $C$, the intended transformation is
$$
F:\ |x\rangle_S|2^j\rangle_M|b\rangle_C \mapsto |x\cdot z_j^{\,b}\rangle_S|2^j\rangle_M|b\rangle_C,
$$
so when $b=0$ nothing happens, and when $b=1$ the selected spin is flipped. This is implemented by a chain of Toffolis, 
$$
F = \Pi_{j=1}^n \mathrm{CCX}(M_j,C;S_j).
$$

For $\ket{m}_M, m = 2^j$ we have:
$$
\begin{align*}
|x\rangle_S|2^j\rangle_M|0\rangle_C & \mapsto |x\rangle_S |2^j\rangle_M|0\rangle_C, \\
|x\rangle_S|2^j\rangle_M|1\rangle_C & \mapsto |x\oplus e_j\rangle_S|2^j\rangle_M|1\rangle_C,
\end{align*}
$$
where $e_j$ is the bitstring with a $1$ only at position $j$ and $\oplus$ is XOR in the $0/1$ encoding of spins. This uses $n$ Toffoli gates with depth $O(n)$.


#### Log depth

For the **log-depth** implementation, the issue is that all $n$ Toffolis share the same coin control $C$, which prevents straightforward parallelization. 

Define an auxiliary register $\ket{\cdot}_A$ of size $N-1$. We use it to create $N-1$ copies of the coin: 
* at time 1 copy $\ket{C}$ into $\ket{A_0}$
* at time 2 copy $\ket{C}$ into $\ket{A_1}$ and $\ket{A_0}$ into $\ket{A_2}$
* each step doubles the amout of coins. Then, one can apply $\Pi_{k=1}^n \mathrm{CCX}(M_k,C_k;S_k)$ all in parallel. We need to uncompute the coin register at the end though.


In [6]:
class MoveAcceptanceF(Gate):
    
    def __init__(self, ising_model: IsingModel, construction: str = "linear", label=None):
        self.ising_model = ising_model
        self.n = int(self.ising_model.n)
        self.construction = construction
        assert self.construction in ["linear", "log"], "The only allowed modes are 'linear' and 'log'"

        self.n_aux_qubits = 0 if self.construction == "linear" else self.n - 1
        super().__init__("F", 2 * self.n + 1 + self.n_aux_qubits, [], label=label)
        self.definition = self._build_definition()

    def _build_definition(self):
        n = self.n

        if self.construction == "linear" or n <= 1:
            qc = QuantumCircuit(2 * n + 1, name="F")
            S = list(range(n))
            M = list(range(n, 2 * n))
            C = 2 * n
            for j in range(n):
                qc.ccx(M[j], C, S[j])
            return qc

        # log-depth: fanout coin C into (n-1) ancillas, use them as parallel coin-controls, then unfanout
        qc = QuantumCircuit(2 * n + 1 + (n - 1), name="F")
        S = list(range(n))
        M = list(range(n, 2 * n))
        C = 2 * n
        A = list(range(2 * n + 1, 2 * n + 1 + (n - 1)))  # coin copies

        raise ValueError("Not implemented yet")

In [7]:
n = example_model.n
F = MoveAcceptanceF(example_model, construction="linear")

S = list(range(n))
M = list(range(n, 2*n))
C = 2*n

def prepare_state(qc, x, j, b):
    # prepare |x>_S
    for k in range(n):
        if (x >> k) & 1:
            qc.x(S[k])
    # prepare unary move |j>_M
    qc.x(M[j])
    # prepare coin |b>
    if b:
        qc.x(C)

# For |b>_C, b = 0 the unitary has no effect
print("Checking the action of F with the coin b=0. The state should remain the same")
for x in range(2**n):
    for j in range(n):
        qc = QuantumCircuit(2*n + 1)
        prepare_state(qc, x, j, 0)
        probs_in = Statevector.from_label("0" * (2*n + 1)).evolve(qc).probabilities_dict()
        qc.append(F, S+M+[C])
        probs_out = Statevector.from_label("0" * (2*n + 1)).evolve(qc).probabilities_dict()
        print("CORRECT?", "Y" if probs_in == probs_out else "N", " | ", probs_in, "=>", probs_out)

# For |b>_C, b = 1 should flip the j-th bit
print("\nChecking the action of F with the coin b=1. The j-th bit should be flipped")
for x in range(2**n):
    for j in range(n):
        qc = QuantumCircuit(2*n + 1)
        prepare_state(qc, x, j, 1)
        probs_in = Statevector.from_label("0" * (2*n + 1)).evolve(qc).probabilities_dict()
        bitstring_in = int(list(probs_in.keys())[0], base=2)
        qc.append(F, S+M+[C])
        probs_out = Statevector.from_label("0" * (2*n + 1)).evolve(qc).probabilities_dict()
        bitstring_out = int(list(probs_out.keys())[0], base=2)
        print("CORRECT?", "Y" if bitstring_in == bitstring_out ^ (1 << j) else "N", " | ", probs_in, "=>", probs_out)

Checking the action of F with the coin b=0. The state should remain the same
CORRECT? Y  |  {np.str_('000010000'): np.float64(1.0)} => {np.str_('000010000'): np.float64(1.0)}
CORRECT? Y  |  {np.str_('000100000'): np.float64(1.0)} => {np.str_('000100000'): np.float64(1.0)}
CORRECT? Y  |  {np.str_('001000000'): np.float64(1.0)} => {np.str_('001000000'): np.float64(1.0)}
CORRECT? Y  |  {np.str_('010000000'): np.float64(1.0)} => {np.str_('010000000'): np.float64(1.0)}
CORRECT? Y  |  {np.str_('000010001'): np.float64(1.0)} => {np.str_('000010001'): np.float64(1.0)}
CORRECT? Y  |  {np.str_('000100001'): np.float64(1.0)} => {np.str_('000100001'): np.float64(1.0)}
CORRECT? Y  |  {np.str_('001000001'): np.float64(1.0)} => {np.str_('001000001'): np.float64(1.0)}
CORRECT? Y  |  {np.str_('010000001'): np.float64(1.0)} => {np.str_('010000001'): np.float64(1.0)}
CORRECT? Y  |  {np.str_('000010010'): np.float64(1.0)} => {np.str_('000010010'): np.float64(1.0)}
CORRECT? Y  |  {np.str_('000100010'): np.

### Unitary $R$ (reflection on move=0 and coin=0)

This one admits two implementations, linear and log depth, too. Here's the linear implementation only. 

In [8]:
class Reflection00(Gate):
    def __init__(self, ising_model: IsingModel, label=None):
        self.ising_model = ising_model
        self.N = int(self.ising_model.n)          # unary Move register size
        if self.N < 0:
            raise ValueError("n must be >= 0")
        super().__init__("R00", self.N + 1, [], label=label)  # (M of size N) + (coin C)
        self.definition = self._build_definition()

    def _build_definition(self):
        qc = QuantumCircuit(self.N + 1, name="R00")
        if self.N == 0:
            qc.z(0)
            qc.global_phase += np.pi             # diag(-1,+1)
            return qc
        qs = list(range(self.N + 1))
        qc.x(qs)
        qc.mcp(np.pi, qs[:-1], qs[-1])           # -1 on |1...1>
        qc.x(qs)                                 # -> -1 on |0...0>
        return qc

In [9]:
R00 = Reflection00(example_model)
U = Operator(R00).data

dim = 2 ** (example_model.n + 1)
U_expected = np.eye(dim, dtype=complex)
U_expected[0, 0] = -1.0                         # flip phase only on |0...0>_(M,C)

print(np.allclose(U, U_expected))

True


### Unitary $B$ (Boltzmann coin)

Here we have a main implementation in Sec 2.4 that I can follow, and is the one presented here. I do not fully understand the QSP-based approach, how  they expect to implement $S_1$ (pag 2 second col)?

Recall:
1. Let $|N_j|=t$ (in our example $|N_j| \le 1 + d(k-1) = 1 + 3(2-1) = 4$)
2. Enumerate all $2^t$ assignments $u\in\{0,1\}^t$ of the spins in $N_j$
3. For each $u$, precompute classically the corresponding angle $\theta_{u,j}=\arcsin\!\sqrt{\min\!\left(1,e^{-\beta\Delta_j(u)}\right)}$
4. For each $u$, apply the controlled $R_y$
$$
\text{if }(M=j)\text{ and }(x|_{N_j}=u)\text{ then apply }R_y(2\theta_{u,j})\text{ on }C.
$$

In [10]:
class BoltzmannCoinRj(Gate):
    def __init__(self, ising_model: IsingModel, beta: float, j: int, label=None):
        self.ising_model = ising_model
        self.beta = float(beta)
        self.j = int(j)
        self.n = ising_model.n
        assert 0 <= j < self.n, f"The move index 0 <= j < n is invalid, {j=}"
        super().__init__(f"R{self.j}", 2 * self.n + 1, [], label=label)
        self.definition = self._build_definition()

    def _build_definition(self):
        n = self.n
        qc = QuantumCircuit(2 * n + 1, name=f"R{self.j}")
        S = list(range(n))
        M = list(range(n, 2 * n))
        C = 2 * n

        # Step 1: determine the neighbors of spin j in the terms of the model
        Nj = self._find_neighbors_Nj()
        assert len(Nj) <= 1 + self.ising_model.d * (self.ising_model.k - 1)
        ctrls = [M[self.j]] + [S[q] for q in Nj] # control qubits

        # Step 2: enumerate all the possible assignments of the spins in N_j
        assignments = product([0, 1], repeat=len(Nj))
        
        for bits in assignments:
            # Step 3: precompute classically the corresponding angle
            angle = 2.0 * self._calculate_theta(Nj, bits)
            # Step 4: apply the controlled Ry
            ctrl_state = "".join("1" if b else "0" for b in bits[::-1]) + "1" # reversed as we accomodate qiskit's endianness
            qc.append(RYGate(angle).control(len(ctrls), ctrl_state=ctrl_state), ctrls + [C])

        return qc

    def _find_neighbors_Nj(self):
        Nj = {self.j}
        for om in self.ising_model.Omega:
            if self.j in om:
                Nj.update(om)
        return sorted(Nj)

    def _calculate_theta(self, Nj, bits):
        delta = self._calculate_delta(Nj, bits)
        A = 1.0 if delta <= 0 else min(1.0, float(np.exp(-self.beta * delta)))
        return float(np.arcsin(np.sqrt(A)))

    def _calculate_delta(self, Nj, bits):
        x = np.ones(self.n)
        x[list(Nj)] = 1 - 2*np.array(bits)
        return -2.0 * sum(J * np.prod(x[list(om)]) for J, om in zip(self.ising_model.J, self.ising_model.Omega) if self.j in om)


class BoltzmannCoinB(Gate):
    def __init__(self, model: IsingModel, beta: float, label=None):
        self.model = model
        self.beta = float(beta)
        self.n = int(model.n)
        super().__init__("B", 2 * self.n + 1, [], label=label)
        self.definition = self._build_definition()

    def _build_definition(self):
        qc = QuantumCircuit(2 * self.n + 1, name="B")
        for j in range(self.n):
            qc.append(BoltzmannCoinRj(self.model, self.beta, j), list(range(2 * self.n + 1)))
        return qc


In [11]:
import numpy as np
from qiskit.circuit import QuantumCircuit
from qiskit.quantum_info import Statevector

beta = 1.0
n = example_model.n
S = list(range(n))
M = list(range(n, 2*n))
C = 2*n

def energy(x):
    spins = [1 - 2*((x >> i) & 1) for i in range(n)]  # 0->+1, 1->-1
    return sum(J * np.prod([spins[q] for q in om]) for J, om in zip(example_model.J, example_model.Omega))

def approximate_dict(d, decimals=10):
    return {k: np.round(v, decimals) for k, v in d.items() if v > 1e-10}

# --- 1) If the move in M is not j, then R_j must do nothing (coin stays |0>, S and M unchanged) ---
print("Checking the action of R_j when the move register is m != j (R_j should be identity)")
for j in range(n):
    Rj = BoltzmannCoinRj(example_model, beta, j)
    for x in range(2**n):
        for m in range(n):
            if m == j:
                continue
            qc = QuantumCircuit(2*n + 1)
            prepare_state(qc, x, m, 0)
            probs_in = approximate_dict(Statevector.from_label("0" * (2*n + 1)).evolve(qc).probabilities_dict())
            qc.append(Rj, S + M + [C])
            probs_out = approximate_dict(Statevector.from_label("0" * (2*n + 1)).evolve(qc).probabilities_dict())
            print("CORRECT?", "Y" if probs_in == probs_out else "N", f" | x={x:0{n}b}, j={j}, m={m} | {probs_in} => {probs_out} ")

# --- 2) If the move in M is j and coin starts in |0>, then R_j should load A into the coin ---
print("\nChecking the action of R_j when the move register is m = j (coin probability should match MH acceptance)")
for j in range(n):
    Rj = BoltzmannCoinRj(example_model, beta, j)
    for x in range(2**n):
        qc = QuantumCircuit(2*n + 1)
        prepare_state(qc, x, j, 0)
        qc.append(Rj, S + M + [C])
        probs_out = Statevector.from_label("0" * (2*n + 1)).evolve(qc).probabilities_dict()

        y = x ^ (1 << j)
        Delta = energy(y) - energy(x)
        A = 1.0 if Delta <= 0 else min(1.0, float(np.exp(-beta * Delta)))

        p1 = sum(p for bs, p in probs_out.items() if ((int(bs, 2) >> C) & 1) == 1)
        ok_prob = np.isclose(p1, A, atol=1e-12)

        base = x | (1 << (n + j))  # system bits + unary move bit, coin masked out
        ok_regs = all(((int(bs, 2) & ~(1 << C)) == base) for bs in probs_out.keys())

        print(
            "CORRECT?", "Y" if (ok_prob and ok_regs) else "N",
            f" | x={x:0{n}b}, j={j}, Δ={Delta:+.6f}, A={A:.12f}, p(C=1)={p1:.12f}"
        )


Checking the action of R_j when the move register is m != j (R_j should be identity)
CORRECT? Y  | x=0000, j=0, m=1 | {np.str_('000100000'): np.float64(1.0)} => {np.str_('000100000'): np.float64(1.0)} 
CORRECT? Y  | x=0000, j=0, m=2 | {np.str_('001000000'): np.float64(1.0)} => {np.str_('001000000'): np.float64(1.0)} 
CORRECT? Y  | x=0000, j=0, m=3 | {np.str_('010000000'): np.float64(1.0)} => {np.str_('010000000'): np.float64(1.0)} 
CORRECT? Y  | x=0001, j=0, m=1 | {np.str_('000100001'): np.float64(1.0)} => {np.str_('000100001'): np.float64(1.0)} 
CORRECT? Y  | x=0001, j=0, m=2 | {np.str_('001000001'): np.float64(1.0)} => {np.str_('001000001'): np.float64(1.0)} 
CORRECT? Y  | x=0001, j=0, m=3 | {np.str_('010000001'): np.float64(1.0)} => {np.str_('010000001'): np.float64(1.0)} 
CORRECT? Y  | x=0010, j=0, m=1 | {np.str_('000100010'): np.float64(1.0)} => {np.str_('000100010'): np.float64(1.0)} 
CORRECT? Y  | x=0010, j=0, m=2 | {np.str_('001000010'): np.float64(1.0)} => {np.str_('001000010'

### Integration test

## Experimental evaluation